In [1]:
import pandas as pd
import numpy as np
import os
import cv2
import glob
import random
import matplotlib.pyplot as plt

from sklearn.cluster import KMeans, spectral_clustering, DBSCAN, FeatureAgglomeration, AffinityPropagation
from sklearn.cluster import Birch, MiniBatchKMeans, MeanShift, SpectralClustering
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn import preprocessing
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, MaxPooling2D, Flatten
from keras.layers import Dropout, Convolution2D

Using TensorFlow backend.


In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Thu Dec 21 13:42:29 2017

@author: Dhara Mungra
"""



faceDet = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
faceDet_two = cv2.CascadeClassifier("haarcascade_frontalface_alt2.xml")
faceDet_three = cv2.CascadeClassifier("haarcascade_frontalface_alt.xml")
faceDet_four = cv2.CascadeClassifier("haarcascade_frontalface_alt_tree.xml")

EMOTIONS = ['angry', 'disgusted', 'fearful', 'happy', 'sad', 'surprised', 'neutral']




from constants import *
import cv2
import pandas as pd
import numpy as np
from PIL import Image
import pickle
import random
SIZE_FACE=48
cascade_classifier = cv2.CascadeClassifier(CASC_PATH)

def format_image(image):
  if len(image.shape) > 2 and image.shape[2] == 3:
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  else:
    image = cv2.imdecode(image, cv2.CV_LOAD_IMAGE_GRAYSCALE)
  gray_border = np.zeros((150, 150), np.uint8)
  gray_border[:,:] = 200
  
  gray_border[int(((150 / 2) - int((SIZE_FACE/2)))):int((150/2)+int((SIZE_FACE/2))), int((150/2)-int((SIZE_FACE/2))):int((150/2)+int((SIZE_FACE/2)))] = image
  image = gray_border

  faces = cascade_classifier.detectMultiScale(
    image,
    scaleFactor = 1.3,
    minNeighbors = 5
  )
  # None is we don't found an image
  if not len(faces) > 0:
    #print "No hay caras"
    return None
  else:
      max_area_face = faces[0]
      for face in faces:
          if face[2] * face[3] > max_area_face[2] * max_area_face[3]:
              max_area_face = face
  # Chop image to face
      face = max_area_face
      image = image[face[1]:(face[1] + face[2]), face[0]:(face[0] + face[3])]
  # Resize image to network size

      try:
          image = cv2.resize(image, (int(SIZE_FACE), int(SIZE_FACE)), interpolation = cv2.INTER_CUBIC) / 255.
      except Exception:
          print("[+] Problem during resize")
          return None
      print (image.shape)
      return image


def emotion_to_vec(x):
    d = np.zeros(len(EMOTIONS))
    d[x] = 1.0
    return d

def flip_image(image):
    return cv2.flip(image, 1)

def data_to_image(data):
    #print data
    data_image = np.fromstring(str(data), dtype = np.uint8, sep = ' ').reshape((int(SIZE_FACE), int(SIZE_FACE)))
    data_image = Image.fromarray(data_image).convert('RGB')
    data_image = np.array(data_image)[:, :, ::-1].copy() 
    data_image = format_image(data_image)
    return data_image

FILE_PATH = 'fer2013.csv'
data = pd.read_csv('fer2013.csv')

labels = []
images = []
index = 1

total = data.shape[0]
for index, row in data.iterrows():
    emotion = emotion_to_vec(row['emotion'])
    
    image = data_to_image(row['pixels'])
    if image is not None:
        labels.append(emotion)
        #image=image.flatten()
        #image.reshape(image.shape[0],-1)
        #image=np.hstack((image,emotion))
        #image=image.concatenate(emotion)
        images.append(image)
        '''
        image = cv2.resize(image, (48, 48)) #Resize face so all images have same size
                #Write image
        
        M = np.float32([[1,0,100],[0,1,50]])
        dst = cv2.warpAffine(image,M,(48,48))
        pts1 = np.float32([[10,10],[245,10],[10,245],[245,245]])
        pts2 = np.float32([[0,0],[255,0],[0,255],[300,255]])
        M = cv2.getPerspectiveTransform(pts1,pts2)
        zoom = cv2.warpPerspective(image,M,(48,48))
        
        rotation_matrix = cv2.getRotationMatrix2D((48/2, 48/2), 20, 1)
        rotation_left = cv2.warpAffine(image, rotation_matrix, (48,48))
        pts3 = np.float32([[25,25],[230,25],[25,230],[230,230]])
        pts4 = np.float32([[0,0],[255,0],[0,255],[300,255]])
        M1 = cv2.getPerspectiveTransform(pts3,pts4)
        rotate_left = cv2.warpPerspective(rotation_left,M1,(48,48))
        rotation_matrix = cv2.getRotationMatrix2D((48/2, 48/2), -20, 1)
        rotation_right = cv2.warpAffine(image, rotation_matrix, (48,48))
        rotate_right = cv2.warpPerspective(rotation_right,M1,(48,48))
        flip_lr = cv2.flip(image, 1)
        flip_tb = cv2.flip(image, 0)
        flip_both = cv2.flip(image, -1)
        
        cv2.resize(flip_lr, (48,48))
        cv2.resize(flip_tb, (48,48))
        cv2.resize(flip_both, (48,48))
        cv2.resize(zoom, (48,48))
        cv2.resize(rotate_left, (48,48))
        cv2.resize(rotate_right, (48,48))
        cv2.resize(image, (48,48))
        images.append(flip_lr)
        labels.append(emotion)
        images.append(flip_tb)
        labels.append(emotion)
        images.append(flip_both)
        labels.append(emotion)
        images.append(zoom)
        labels.append(emotion)
        images.append(rotate_left)
        labels.append(emotion)
        images.append(rotate_right)
        labels.append(emotion)'''
        
        plt.plot(image)
    else:
        print ("Error")
    index += 1
    print ("Progreso: {}/{} {:.2f}%".format(index, total, index * 100.0 / total))
filename="train_image.sav"

print ("Total: " + str(len(images)))
print(images.shape)
np.save('images.npy', images)
np.save('labels.npy', labels)        
#df=pd.DataFrame(images)
#print(df.head())



(48, 48)
Progreso: 1/35887 0.00%
Error
Progreso: 2/35887 0.01%
(48, 48)
Progreso: 3/35887 0.01%
Error
Progreso: 4/35887 0.01%
(48, 48)
Progreso: 5/35887 0.01%
(48, 48)
Progreso: 6/35887 0.02%
Error
Progreso: 7/35887 0.02%
(48, 48)
Progreso: 8/35887 0.02%
(48, 48)
Progreso: 9/35887 0.03%
(48, 48)
Progreso: 10/35887 0.03%
(48, 48)
Progreso: 11/35887 0.03%
Error
Progreso: 12/35887 0.03%
Error
Progreso: 13/35887 0.04%
Error
Progreso: 14/35887 0.04%
(48, 48)
Progreso: 15/35887 0.04%
(48, 48)
Progreso: 16/35887 0.04%
Error
Progreso: 17/35887 0.05%
Error
Progreso: 18/35887 0.05%
Error
Progreso: 19/35887 0.05%
Error
Progreso: 20/35887 0.06%
Error
Progreso: 21/35887 0.06%
Error
Progreso: 22/35887 0.06%
(48, 48)
Progreso: 23/35887 0.06%
Error
Progreso: 24/35887 0.07%
Error
Progreso: 25/35887 0.07%
(48, 48)
Progreso: 26/35887 0.07%
Error
Progreso: 27/35887 0.08%
Error
Progreso: 28/35887 0.08%
Error
Progreso: 29/35887 0.08%
Error
Progreso: 30/35887 0.08%
(48, 48)
Progreso: 31/35887 0.09%
Error
Pro

Progreso: 250/35887 0.70%
(48, 48)
Progreso: 251/35887 0.70%
(48, 48)
Progreso: 252/35887 0.70%
(48, 48)
Progreso: 253/35887 0.70%
Error
Progreso: 254/35887 0.71%
Error
Progreso: 255/35887 0.71%
Error
Progreso: 256/35887 0.71%
Error
Progreso: 257/35887 0.72%
Error
Progreso: 258/35887 0.72%
Error
Progreso: 259/35887 0.72%
Error
Progreso: 260/35887 0.72%
(48, 48)
Progreso: 261/35887 0.73%
Error
Progreso: 262/35887 0.73%
(48, 48)
Progreso: 263/35887 0.73%
(48, 48)
Progreso: 264/35887 0.74%
Error
Progreso: 265/35887 0.74%
(48, 48)
Progreso: 266/35887 0.74%
Error
Progreso: 267/35887 0.74%
Error
Progreso: 268/35887 0.75%
(48, 48)
Progreso: 269/35887 0.75%
(48, 48)
Progreso: 270/35887 0.75%
(48, 48)
Progreso: 271/35887 0.76%
Error
Progreso: 272/35887 0.76%
Error
Progreso: 273/35887 0.76%
(48, 48)
Progreso: 274/35887 0.76%
Error
Progreso: 275/35887 0.77%
Error
Progreso: 276/35887 0.77%
Error
Progreso: 277/35887 0.77%
(48, 48)
Progreso: 278/35887 0.77%
Error
Progreso: 279/35887 0.78%
(48, 48)
P

Progreso: 498/35887 1.39%
Error
Progreso: 499/35887 1.39%
(48, 48)
Progreso: 500/35887 1.39%
Error
Progreso: 501/35887 1.40%
Error
Progreso: 502/35887 1.40%
Error
Progreso: 503/35887 1.40%
(48, 48)
Progreso: 504/35887 1.40%
Error
Progreso: 505/35887 1.41%
Error
Progreso: 506/35887 1.41%
Error
Progreso: 507/35887 1.41%
(48, 48)
Progreso: 508/35887 1.42%
(48, 48)
Progreso: 509/35887 1.42%
(48, 48)
Progreso: 510/35887 1.42%
Error
Progreso: 511/35887 1.42%
Error
Progreso: 512/35887 1.43%
Error
Progreso: 513/35887 1.43%
Error
Progreso: 514/35887 1.43%
Error
Progreso: 515/35887 1.44%
Error
Progreso: 516/35887 1.44%
(48, 48)
Progreso: 517/35887 1.44%
Error
Progreso: 518/35887 1.44%
Error
Progreso: 519/35887 1.45%
Error
Progreso: 520/35887 1.45%
Error
Progreso: 521/35887 1.45%
(48, 48)
Progreso: 522/35887 1.45%
Error
Progreso: 523/35887 1.46%
(48, 48)
Progreso: 524/35887 1.46%
(48, 48)
Progreso: 525/35887 1.46%
(48, 48)
Progreso: 526/35887 1.47%
Error
Progreso: 527/35887 1.47%
(48, 48)
Progres

Progreso: 747/35887 2.08%
(48, 48)
Progreso: 748/35887 2.08%
Error
Progreso: 749/35887 2.09%
Error
Progreso: 750/35887 2.09%
(48, 48)
Progreso: 751/35887 2.09%
(48, 48)
Progreso: 752/35887 2.10%
(48, 48)
Progreso: 753/35887 2.10%
(48, 48)
Progreso: 754/35887 2.10%
Error
Progreso: 755/35887 2.10%
(48, 48)
Progreso: 756/35887 2.11%
Error
Progreso: 757/35887 2.11%
(48, 48)
Progreso: 758/35887 2.11%
Error
Progreso: 759/35887 2.11%
Error
Progreso: 760/35887 2.12%
(48, 48)
Progreso: 761/35887 2.12%
Error
Progreso: 762/35887 2.12%
Error
Progreso: 763/35887 2.13%
Error
Progreso: 764/35887 2.13%
Error
Progreso: 765/35887 2.13%
Error
Progreso: 766/35887 2.13%
Error
Progreso: 767/35887 2.14%
Error
Progreso: 768/35887 2.14%
Error
Progreso: 769/35887 2.14%
(48, 48)
Progreso: 770/35887 2.15%
Error
Progreso: 771/35887 2.15%
Error
Progreso: 772/35887 2.15%
(48, 48)
Progreso: 773/35887 2.15%
(48, 48)
Progreso: 774/35887 2.16%
Error
Progreso: 775/35887 2.16%
Error
Progreso: 776/35887 2.16%
Error
Progres

Progreso: 995/35887 2.77%
(48, 48)
Progreso: 996/35887 2.78%
Error
Progreso: 997/35887 2.78%
(48, 48)
Progreso: 998/35887 2.78%
Error
Progreso: 999/35887 2.78%
Error
Progreso: 1000/35887 2.79%
(48, 48)
Progreso: 1001/35887 2.79%
Error
Progreso: 1002/35887 2.79%
(48, 48)
Progreso: 1003/35887 2.79%
(48, 48)
Progreso: 1004/35887 2.80%
Error
Progreso: 1005/35887 2.80%
Error
Progreso: 1006/35887 2.80%
Error
Progreso: 1007/35887 2.81%
(48, 48)
Progreso: 1008/35887 2.81%
Error
Progreso: 1009/35887 2.81%
(48, 48)
Progreso: 1010/35887 2.81%
Error
Progreso: 1011/35887 2.82%
Error
Progreso: 1012/35887 2.82%
Error
Progreso: 1013/35887 2.82%
Error
Progreso: 1014/35887 2.83%
Error
Progreso: 1015/35887 2.83%
Error
Progreso: 1016/35887 2.83%
Error
Progreso: 1017/35887 2.83%
(48, 48)
Progreso: 1018/35887 2.84%
(48, 48)
Progreso: 1019/35887 2.84%
Error
Progreso: 1020/35887 2.84%
Error
Progreso: 1021/35887 2.85%
(48, 48)
Progreso: 1022/35887 2.85%
Error
Progreso: 1023/35887 2.85%
Error
Progreso: 1024/358

...

Progreso: 8585/35887 23.92%
(48, 48)
Progreso: 8586/35887 23.93%
(48, 48)
Progreso: 8587/35887 23.93%
Error
Progreso: 8588/35887 23.93%
Error
Progreso: 8589/35887 23.93%
(48, 48)
Progreso: 8590/35887 23.94%
Error
Progreso: 8591/35887 23.94%
(48, 48)
Progreso: 8592/35887 23.94%
(48, 48)
Progreso: 8593/35887 23.94%
Error
Progreso: 8594/35887 23.95%
(48, 48)
Progreso: 8595/35887 23.95%
Error
Progreso: 8596/35887 23.95%
(48, 48)
Progreso: 8597/35887 23.96%
Error
Progreso: 8598/35887 23.96%
Error
Progreso: 8599/35887 23.96%
(48, 48)
Progreso: 8600/35887 23.96%
Error
Progreso: 8601/35887 23.97%
(48, 48)
Progreso: 8602/35887 23.97%
Error
Progreso: 8603/35887 23.97%
(48, 48)
Progreso: 8604/35887 23.98%
Error
Progreso: 8605/35887 23.98%
(48, 48)
Progreso: 8606/35887 23.98%
Error
Progreso: 8607/35887 23.98%
Error
Progreso: 8608/35887 23.99%
Error
Progreso: 8609/35887 23.99%
Error
Progreso: 8610/35887 23.99%
Error
Progreso: 8611/35887 23.99%
(48, 48)
Progreso: 8612/35887 24.00%
(48, 48)
Progreso:

Progreso: 8819/35887 24.57%
Error
Progreso: 8820/35887 24.58%
(48, 48)
Progreso: 8821/35887 24.58%
Error
Progreso: 8822/35887 24.58%
Error
Progreso: 8823/35887 24.59%
Error
Progreso: 8824/35887 24.59%
Error
Progreso: 8825/35887 24.59%
Error
Progreso: 8826/35887 24.59%
Error
Progreso: 8827/35887 24.60%
Error
Progreso: 8828/35887 24.60%
Error
Progreso: 8829/35887 24.60%
(48, 48)
Progreso: 8830/35887 24.61%
Error
Progreso: 8831/35887 24.61%
(48, 48)
Progreso: 8832/35887 24.61%
(48, 48)
Progreso: 8833/35887 24.61%
Error
Progreso: 8834/35887 24.62%
Error
Progreso: 8835/35887 24.62%
Error
Progreso: 8836/35887 24.62%
(48, 48)
Progreso: 8837/35887 24.62%
(48, 48)
Progreso: 8838/35887 24.63%
(48, 48)
Progreso: 8839/35887 24.63%
Error
Progreso: 8840/35887 24.63%
Error
Progreso: 8841/35887 24.64%
(48, 48)
Progreso: 8842/35887 24.64%
Error
Progreso: 8843/35887 24.64%
(48, 48)
Progreso: 8844/35887 24.64%
Error
Progreso: 8845/35887 24.65%
Error
Progreso: 8846/35887 24.65%
Error
Progreso: 8847/35887 

Progreso: 9054/35887 25.23%
Error
Progreso: 9055/35887 25.23%
Error
Progreso: 9056/35887 25.23%
Error
Progreso: 9057/35887 25.24%
Error
Progreso: 9058/35887 25.24%
Error
Progreso: 9059/35887 25.24%
Error
Progreso: 9060/35887 25.25%
Error
Progreso: 9061/35887 25.25%
(48, 48)
Progreso: 9062/35887 25.25%
(48, 48)
Progreso: 9063/35887 25.25%
Error
Progreso: 9064/35887 25.26%
Error
Progreso: 9065/35887 25.26%
Error
Progreso: 9066/35887 25.26%
(48, 48)
Progreso: 9067/35887 25.27%
Error
Progreso: 9068/35887 25.27%
(48, 48)
Progreso: 9069/35887 25.27%
(48, 48)
Progreso: 9070/35887 25.27%
Error
Progreso: 9071/35887 25.28%
Error
Progreso: 9072/35887 25.28%
(48, 48)
Progreso: 9073/35887 25.28%
Error
Progreso: 9074/35887 25.28%
Error
Progreso: 9075/35887 25.29%
(48, 48)
Progreso: 9076/35887 25.29%
(48, 48)
Progreso: 9077/35887 25.29%
(48, 48)
Progreso: 9078/35887 25.30%
Error
Progreso: 9079/35887 25.30%
Error
Progreso: 9080/35887 25.30%
Error
Progreso: 9081/35887 25.30%
(48, 48)
Progreso: 9082/358

Progreso: 9288/35887 25.88%
Error
Progreso: 9289/35887 25.88%
Error
Progreso: 9290/35887 25.89%
Error
Progreso: 9291/35887 25.89%
Error
Progreso: 9292/35887 25.89%
(48, 48)
Progreso: 9293/35887 25.90%
Error
Progreso: 9294/35887 25.90%
(48, 48)
Progreso: 9295/35887 25.90%
Error
Progreso: 9296/35887 25.90%
(48, 48)
Progreso: 9297/35887 25.91%
(48, 48)
Progreso: 9298/35887 25.91%
(48, 48)
Progreso: 9299/35887 25.91%
Error
Progreso: 9300/35887 25.91%
(48, 48)
Progreso: 9301/35887 25.92%
(48, 48)
Progreso: 9302/35887 25.92%
Error
Progreso: 9303/35887 25.92%
(48, 48)
Progreso: 9304/35887 25.93%
Error
Progreso: 9305/35887 25.93%
Error
Progreso: 9306/35887 25.93%
Error
Progreso: 9307/35887 25.93%
Error
Progreso: 9308/35887 25.94%
(48, 48)
Progreso: 9309/35887 25.94%
(48, 48)
Progreso: 9310/35887 25.94%
(48, 48)
Progreso: 9311/35887 25.95%
Error
Progreso: 9312/35887 25.95%
Error
Progreso: 9313/35887 25.95%
Error
Progreso: 9314/35887 25.95%
Error
Progreso: 9315/35887 25.96%
Error
Progreso: 9316/

Progreso: 9524/35887 26.54%
Error
Progreso: 9525/35887 26.54%
(48, 48)
Progreso: 9526/35887 26.54%
Error
Progreso: 9527/35887 26.55%
Error
Progreso: 9528/35887 26.55%
Error
Progreso: 9529/35887 26.55%
Error
Progreso: 9530/35887 26.56%
Error
Progreso: 9531/35887 26.56%
Error
Progreso: 9532/35887 26.56%
Error
Progreso: 9533/35887 26.56%
Error
Progreso: 9534/35887 26.57%
Error
Progreso: 9535/35887 26.57%
Error
Progreso: 9536/35887 26.57%
(48, 48)
Progreso: 9537/35887 26.58%
(48, 48)
Progreso: 9538/35887 26.58%
Error
Progreso: 9539/35887 26.58%
(48, 48)
Progreso: 9540/35887 26.58%
(48, 48)
Progreso: 9541/35887 26.59%
Error
Progreso: 9542/35887 26.59%
Error
Progreso: 9543/35887 26.59%
(48, 48)
Progreso: 9544/35887 26.59%
(48, 48)
Progreso: 9545/35887 26.60%
(48, 48)
Progreso: 9546/35887 26.60%
Error
Progreso: 9547/35887 26.60%
(48, 48)
Progreso: 9548/35887 26.61%
(48, 48)
Progreso: 9549/35887 26.61%
(48, 48)
Progreso: 9550/35887 26.61%
Error
Progreso: 9551/35887 26.61%
Error
Progreso: 9552/

In [ ]:
from sklearn.cross_validation import train_test_split
imagetrain, imagetest, labeltrain, labeltest = train_test_split(images, labels, test_size = 0.20, random_state = 1)
from sklearn.cross_validation import train_test_split
imagetest, imageval,label_test,label_val = train_test_split(imagetest,labeltest, test_size = 0.50, random_state = 0)
#pickle.dump(image_train, open(filename, 'wb'))
np.save('train_set.npy', image_train1)
#df.to_csv('train.csv',index=False)
np.save('train_set.npy', imagetrain)
np.save('train_label.npy', labeltrain)
np.save('test_set.npy', imagetest)
np.save('test_label.npy', labeltest)
np.save('val_set.npy', imageval)
np.save('val_label.npy', labelval)

In [85]:
CASC_PATH = './haarcascade_files/haarcascade_frontalface_default.xml'
SIZE_FACE = 48
#TOTAL = 22428
emotions = ["neutral", "anger", "contempt", "disgust", "fear", "happy", "sadness", "surprise"] #Define emotion orderSAVE_DIRECTORY = './'
SAVE_MODEL_FILENAME = 'saved_model'

print("hiii")

hiii


In [81]:

import numpy as np
from os.path import join
import numpy as np
SIZE_FACE=48
import cv2

imagetrain= np.load("train_set.npy")
labeltrain= np.load("train_label.npy")
imagetest = np.load("test_set.npy")
labeltest = np.load("test_label.npy")
imageval = np.load("val_set.npy")
labelval = np.load("val_label.npy")
#imageval, imagetest, labelval, labeltest = train_test_split(imagetest, labeltest, test_size=0.5, random_state=None)
print("images")
print(labelval.shape)
print(labeltest.shape)
imagetrain= imagetrain.reshape([-1, SIZE_FACE, SIZE_FACE, 1])
imagetest = imagetest.reshape([-1, SIZE_FACE, SIZE_FACE, 1])
imageval = imageval.reshape([-1, SIZE_FACE, SIZE_FACE, 1])
labeltrain = labeltrain.reshape([-1, len(emotions)])
labeltest = labeltest.reshape([-1, len(emotions)])
labelval = labelval.reshape([-1, len(emotions)])
print(imagetrain.shape)
print("hii")

hii


In [ ]:
import cv2
plt.imshow(cv2.resize(imagetrain[100], (48,48)))
plt.show()

In [ ]:
import cv2
X = []
for i in range(len(imagetrain)):
    X[i] = cv2.resize(imagetrain[i], (64,64))

# print(X.shape)

np.save("train_set.npy", X)

In [ ]:
model = Sequential()

model.add(Convolution2D(32, (3, 3), activation='relu', input_shape=(64,64,1), padding='same'))

model.add(Convolution2D(32, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.7))
model.add(Convolution2D(64, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.7))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(8, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(imagetrain, labeltrain, batch_size=64, epochs=30, verbose=1, validation_data=(imageval,labelval))
scores = model.evaluate(imagetest, labeltest, verbose=0)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

[!] Usage: python emotion_recognition.py
	 emotion_recognition.py train 	 Trains and saves model with saved dataset
	 emotion_recognition.py poc 	 Launch the proof of concept
